# N Smallest Capitalization Stock

1. Set the number of stocks want to trade, N.
1. 设定好要交易的股票数量stocksnum

2. Find the first N number of stocks with the smallest market capitalization.
2. 每天找出市值排名最小的前stocksnum只股票作为要买入的股票。

3. Sell the stock if the price of the current stock holding is not small enough anymore (no in the list of N stocks)
3. 若已持有的股票的市值已经不够小而不在要买入的股票中，则卖出这些股票。

4. Buy the new stocks from the current list of N stocks with the smallest market capitalization.
4. 买入要买入的股票，买入金额为当前可用资金的stocksnum分之一。

In [1]:
import pandas as pd
import numpy as np

In [164]:
# Read in the sample data csv  (use df scraped from yahoo finance)
# Use Close and Volume value to get daily market captalization for each stock 
# Market Cap = Close * Volume for each stock
df = pd.read_csv('SampleData.csv')
columns = df.columns.tolist()

for x in columns[1:26]:
    index = columns.index(x)
    if index < 26:
        market = df[df.columns[index]] * df[df.columns[index+25]]
        df[str(x) + '_' + 'Mkt'] = market


# Make the 'Date' row index
df = df.set_index('Date')

In [59]:
# Only capture the Market Cap as needed
# [:, 50:76] Depends on the dataset (50:76) numbers need to change
df_small = df.iloc[:, 50:76]
small = df_small.T

In [63]:
# small1 = small.sort_values([small.columns[0]])
# list1 = np.array(small1.index[0:5]).tolist()
# small2 = small.sort_values([small.columns[1]])
# list2 = np.array(small2.index[0:5]).tolist()
#print(list1)
#print(small1)
#print(small2)
# a = []
# a.append(list1)
# a.append(list2)
# print(a)

In [187]:
def initial_list(x, cashpool, stocknum, y):
    
    #### Get the Initial Basket For date X ####
    position = small.columns.tolist().index(x) # get the column index of the date x input
    small1 = small.sort_values([x]) # sort according to date x
    choice = np.array(small1.index[0:stocknum]).tolist() # get the smallest N stocks of the day
    #ones = np.ones(stocknum, dtype=int) # determine no of shares to buy for each stock
    #a = dict(zip(choice, ones)) 
    
    # get the column name I want to find the close price for each stock on day x
    price = [i.split('_')[0]+"_"+i.split('_')[1] for i in choice]
    
    # find the close price for each stock on day x
    close = []
    for i in range(len(price)):
        b = df.loc[x, price[i]]
        close.append(b)
        invest = cashpool / stocknum
        share = [invest / i for i in close]
        buy_dict = dict(zip(choice, share)) # update buy_list for INITIAL day x
    
    
    ### Get the buy list for date y ###
    position2 = small.columns.tolist().index(y) # get the column index of the date y input
    small2 = small.sort_values([y]) # sort according to date y
    choice2 = np.array(small2.index[0:stocknum]).tolist() # get the smallest N stocks of the day
    print(choice2)


In [188]:
initial_list('2016-11-21', 100, 5, '2018-11-20')

['LYG_Close_Mkt', 'ABEV_Close_Mkt', 'AMRN_Close_Mkt', 'SWN_Close_Mkt', 'CHK_Close_Mkt']


In [184]:
def initial_list(x, cashpool, stocknum,):
    position = small.columns.tolist().index(x) # get the column index of the date x input
    small1 = small.sort_values([x]) # sort according to date x
    choice = np.array(small1.index[0:stocknum]).tolist() # get the smallest N(5) stocks of the day
    #ones = np.ones(stocknum, dtype=int) # determine no of shares to buy for each stock
    #a = dict(zip(choice, ones)) 
    
    # get the column name I want to find the close price for each stock on day x
    price = [i.split('_')[0]+"_"+i.split('_')[1] for i in choice]
    
    # find the close price for each stock on day x
    close = []
    for i in range(len(price)):
        b = df.loc[x, price[i]]
        close.append(b)
        invest = cashpool / stocknum
        share = [invest / i for i in close]
        buy_dict = dict(zip(choice, share)) # update buy_list for INITIAL day x
     
    return (buy_dict)

In [185]:
initial_list('2016-11-21',100, 5)

{'AMRN_Close_Mkt': 5.763688760806916,
 'LYG_Close_Mkt': 6.68896321070234,
 'ABEV_Close_Mkt': 3.8610038610038613,
 'PCG_Close_Mkt': 0.3380662609871535,
 'SIRI_Close_Mkt': 4.3763676148796495}

In [186]:
initial_list('2018-11-20',100, 5)

{'LYG_Close_Mkt': 7.142857142857143,
 'ABEV_Close_Mkt': 4.784688995215311,
 'AMRN_Close_Mkt': 1.154068090017311,
 'SWN_Close_Mkt': 3.9603960396039604,
 'CHK_Close_Mkt': 6.25}

In [ ]:
# 'portfolio' is same as 'buy' on the first day(buy top 5 stocks whose market values are smallest)
portfolio_list = [buy[0]]
for i in range(len(buy) - 1):
    dictMerged = dict(buy[i], **buy[i+1]) # merge dictionarys
    for j in buy[i]:
        if j in buy[i+1]: # if stock A exists in the both portfolios, we set it to be 0 in the merged portfolio
            dictMerged[j] = 0 # 0 means no actions

        else:            # if stock A exists in the first portfolio but not in the second portfolio, we set it to be -1 in the merged portfolio
            dictMerged[j] = -1 # -1 means shorting the stock

                         # if stock A exists in the second portfolio but not in the first portfolio, we set it to be 1 in the merged portfolio
                         # 1 means longing the stock(same as 'buy')

    portfolio_list.append(dictMerged)

In [41]:
# # sort and choose the smallest N -(5) stock for each day
# # Create a large list including buy lists for ALL DAYS
# buy = []
# for i in range (1, len(small.columns)):
#     a = small
#     small1 = a.sort_values([a.columns[i]])
#     ones = np.ones(5, dtype=int)
#     choice = np.array(small1.index[0:5]).tolist()
#     c = dict(zip(choice, ones))
#     buy.append(c)

In [85]:
def buy_list(x):
    position = small.columns.tolist().index(x)
    
    buy = []
    for i in range (len(small.columns)):
        small1 = small.sort_values([small.columns[i]]) #sort according to date i
        choice = np.array(small1.index[0:5]).tolist()  # get the smallest N(5) stocks of the day    
        
        
        
        
        
        ones = np.ones(5, dtype=int) # determine no of shares to buy for each stock
        c = dict(zip(choice, ones)) #combine shares with stocks to a dictionary
        buy.append(c)
    
    return buy[position]

In [57]:
# create portfolio_list for calculating returns
def portfolio_of_date(x):
    position = small.columns.tolist().index(x)
    
    buy = []
    for i in range (len(small.columns)):
        small1 = small.sort_values([a.columns[i]])
        ones = np.ones(5, dtype=int)
        choice = np.array(small1.index[0:5]).tolist()
        c = dict(zip(choice, ones))
        buy.append(c)
    
    # 'portfolio' is same as 'buy' on the first day(buy top 5 stocks whose market values are smallest)
    portfolio_list = [buy[0]]
    for i in range(len(buy) - 1):
        dictMerged = dict(buy[i], **buy[i+1]) # merge dictionarys
        for j in buy[i]:
            if j in buy[i+1]: # if stock A exists in the both portfolios, we set it to be 0 in the merged portfolio
                dictMerged[j] = 0 # 0 means no actions
                
            else:            # if stock A exists in the first portfolio but not in the second portfolio, we set it to be -1 in the merged portfolio
                dictMerged[j] = -1 # -1 means shorting the stock
                
                             # if stock A exists in the second portfolio but not in the first portfolio, we set it to be 1 in the merged portfolio
                             # 1 means longing the stock(same as 'buy')
                             
        portfolio_list.append(dictMerged)
    return portfolio_list[position] # getting portfolio_list during the trading days

In [64]:
a = portfolio_of_date('2016-11-28')

In [36]:

position = small.columns.tolist().index(x)

buy = []
for i in range (1, len(small.columns)):
    small1 = small.sort_values([a.columns[i]])
    ones = np.ones(5, dtype=int)
    choice = np.array(small1.index[0:5]).tolist()
    c = dict(zip(choice, ones))
    buy.append(c)

# 'portfolio' is same as 'buy' on the first day(buy top 5 stocks whose market values are smallest)
portfolio_list = [buy[0]]
for i in range(len(buy) - 1):
    dictMerged = dict(buy[i], **buy[i+1]) # merge dictionarys
    for j in buy[i]:
        if j in buy[i+1]: # if stock A exists in the both portfolios, we set it to be 0 in the merged portfolio
            dictMerged[j] = 0 # 0 means no actions

        else:            # if stock A exists in the first portfolio but not in the second portfolio, we set it to be -1 in the merged portfolio
            dictMerged[j] = -1 # -1 means shorting the stock

                         # if stock A exists in the second portfolio but not in the first portfolio, we set it to be 1 in the merged portfolio
                         # 1 means longing the stock(same as 'buy')

    portfolio_list.append(dictMerged)
portfolio_list[position]

IndentationError: unexpected indent (<ipython-input-36-3fa9f1e15b60>, line 2)

In [146]:
a = {'AMRN_Close_Mkt': 1,
 'LYG_Close_Mkt': 1,
 'ABEV_Close_Mkt': 1,
 'PCG_Close_Mkt': 1,
 'SIRI_Close_Mkt': 1}

pricename = [x.split('_')[0]+"_"+x.split('_')[1] for x in a.keys()]
print(price)

close = []
for i in range(len(price)):
    b = df.loc['2016-11-28', price[i]]
    close.append(b)
    share = [invest / stocknum / i for i in close]
close


['AMRN_Close', 'LYG_Close', 'ABEV_Close', 'PCG_Close', 'SIRI_Close']


[3.34, 2.87, 5.11, 60.740002, 4.54]

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('expand_frame_repr', False)

#open, high, low, close, volumn, adjVolumn, date, MarketCap

#import dataset from Yahoo_finance 导入数据
stock_data = pd.read_csv('',#usecols = ['price','date',] encoding = 'gbk'--read chinese)
stock_data.columns = [i.encode('utf8') for i in stock_data.columns]
stock_data['交易日期'] = pd.to_datetime(stock_data['交易日期'])

# 1. 设定好要交易的股票数量stocksnum
stocknum = 30

# 2. 每天找出市值排名最小的前stocksnum只股票作为要买入的股票。
#Sort the stocks by market cap and date
stock_data.sort_values(by=['date','marketcap', #inplace = True])

holding_first=holding_first.sort(columns='negMarketValue')[0:n]  


#开始的时间太早？
stock_data = stock_data[stock_data['date'] > pd.to_datetime('2016-11-01')]

#删去月末停牌？最后一天不交易，不能买入
stock_data = stock_data[stock_data['是否交易'] != 0]

#该月交易时间过短，去除?
stock_data = stock_data[stock_data['交易天数'] >= 10]

#计算所以股票在下个月的平均涨幅
output = pd.DataFrame()
output['所有股票下个月的涨幅'] = stock_data.groupby('交易日期')['下月涨幅'].mean()

#计算每月市值排名
stock_data['市值_排名'] = stock_data.groupby('交易日期')['总市值'].rank()

#Pick the top 30 stocks -- lowest market cap
stock_data = stock_data['市值_排名'] <= 30



#create a list that contains the top 30 stocks.
buylist = []
    for i in range(30):
              



# 3. 若已持有的股票的市值已经不够小而不在要买入的股票中，则卖出这些股票。

# 4. 买入要买入的股票，买入金额为当前可用资金的stocksnum分之一。
